import numpy as np
xtrain = np.load('traindata.npy',allow_pickle=True)

for i in range(18662):
    if xtrain[i][:,64:92].sum(axis=0).sum() != 0:
        print(i, 'with label', df_train['Label'][i], 'is not equal')

"""
def load_train_dataframe(id):
    train_data = np.load("train/train/{}.npy".format(id))
    return pd.DataFrame(data=train_data)
def load_test_dataframe(id):
    test_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=test_data)

# load training
traindata = []
for id in df_train['Id']:
    dfi = load_train_dataframe(id)
    traindata.append(dfi.values)
print('Done train')
# load test
testdata = []
for id in df_test['Id']:
    dfi = load_test_dataframe(id)
    testdata.append(dfi.values)
print('Done test')

np.save('traindata.npy', traindata)

np.save('testdata.npy', testdata)

"""

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

print('Done')
df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('sample_solution.csv')

print('training data: ',df_train.shape,'\ntest data: ', df_test.shape)

Done
training data:  (18662, 2) 
test data:  (6051, 2)


In [2]:
def process(dfs):
    data = []
    length = 12
    for df in dfs:
        out = np.zeros((df.shape[0],length))
#         out[:,0:length] = df[:,0:8] #1
#         out[:,0:length] = df[:,8:12] #2
#         out[:,0:length] = df[:,12:28] #3
#         out[:,0:length] = df[:,28:44] #4
#         out[:,0:length] = df[:,44:52] #5
#         out[:,0:length] = df[:,52:64] #6
#         out[:,0:length] = df[:,64:80] #7
#         out[:,0:length] = df[:,80:92] #8
#         out[:,0:length] = df[:,92:102] #9

        data.append(out)                 
    return data

def pad_data(dfs):
    padsize = 1000
    data = []
    for df in dfs:
        diff = padsize-df.shape[0]
        if diff >0:
            df = np.pad(df, [(0, diff), (0,0)], 'constant')
        else:
            df = df[-padsize:]
        data.append(df)
        
    data = np.stack(data)
    return data

# Load data + Process + Pad
xtrain = np.load('traindata.npy',allow_pickle=True)
xtrain = process(xtrain)
xtrain = pad_data(xtrain)
# xtest = np.load('testdata.npy',allow_pickle=True)

In [3]:
ytrain = np.array(df_train['Label'])
print('Done')

Done


In [4]:
xtrain.shape

(18662, 1000, 12)

In [5]:
from sklearn.model_selection import train_test_split

xtrain, X_val, ytrain, y_val = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)

import matplotlib.pyplot as plt
def plot_metrics(history):
    metrics =  ['loss', 'auc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8,1])
        else:
            plt.ylim([0,1])

        plt.legend()
        
import sklearn
def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.xlim([-0.5,20])
    plt.ylim([80,100.5])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [6]:
import tensorflow as tf
from tensorflow import keras

METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc'),
]

inputs = keras.Input(shape=(xtrain.shape[1], xtrain.shape[2])) 
x = keras.layers.Conv1D(filters=6, kernel_size=5, padding='same', activation='relu')(inputs)
x = keras.layers.MaxPooling1D(pool_size=2)(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Conv1D(filters=16, kernel_size=5, padding='same', activation='relu')(x)
x = keras.layers.MaxPooling1D(pool_size=2)(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(120, activation='relu')(x)
x = keras.layers.Dense(84, activation='relu')(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=x)

# model.compile(loss='categorical_crossentropy',
#              optimizer='adam',
#              metrics=['accuracy'])

model.compile(
      optimizer=keras.optimizers.Adam(),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=METRICS)

# Take a look at the model summary
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 12)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1000, 6)           366       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 500, 6)            0         
_________________________________________________________________
dropout (Dropout)            (None, 500, 6)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 16)           496       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 16)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 250, 16)           0     

In [7]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)


In [8]:
from tensorflow.keras.utils import to_categorical

EPOCHS = 100
baseline_history = model.fit(
    xtrain,
    ytrain,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(X_val, y_val))

# model.fit(xtrain, to_categorical(ytrain), epochs=30)

Train on 14929 samples, validate on 3733 samples
Epoch 1/100
14929/14929 [==============================] - 13s 849us/sample - loss: 0.6820 - accuracy: 0.5639 - auc: 0.5161 - val_loss: 0.6805 - val_accuracy: 0.5631 - val_auc: 0.5154
Epoch 2/100
14929/14929 [==============================] - 11s 762us/sample - loss: 0.6762 - accuracy: 0.5651 - auc: 0.5283 - val_loss: 0.6789 - val_accuracy: 0.5620 - val_auc: 0.5170
Epoch 3/100
14929/14929 [==============================] - 12s 775us/sample - loss: 0.6743 - accuracy: 0.5657 - auc: 0.5216 - val_loss: 0.6808 - val_accuracy: 0.5631 - val_auc: 0.5168
Epoch 4/100
14929/14929 [==============================] - 12s 784us/sample - loss: 0.6711 - accuracy: 0.5680 - auc: 0.5195 - val_loss: 0.6790 - val_accuracy: 0.5612 - val_auc: 0.5178
Epoch 5/100
14929/14929 [==============================] - 11s 742us/sample - loss: 0.6689 - accuracy: 0.5692 - auc: 0.5343 - val_loss: 0.6816 - val_accuracy: 0.5628 - val_auc: 0.5213
Epoch 6/100
14929/14929 [======

In [9]:
# Load data + Process + Pad
xtest = np.load('testdata.npy',allow_pickle=True)
xtest = process(xtest)
xtest = pad_data(xtest)

ytest = model.predict(xtest)
print('Done')

Done


In [10]:
df_test['Predicted'] = ytest[:, 0]
df_test

,Id,Predicted
0,0,0.549826
1,1,0.549826
2,2,0.549826
3,3,0.549826
4,4,0.549826
5,5,0.549826
6,6,0.549826
7,7,0.549826
8,8,0.549826
9,9,0.549826


In [11]:
df_test.to_csv('8_full.csv', index=False) # Better to be global onehot